# Método de Estimación de Dos Etapas

Maddala, G. S. (1983). Limited-dependent and qualitative variables in econometrics. https://doi.org/10.1017/cbo9780511810176 

### Estimación de Dos Etapas del modelo Heckman


Consideremos el modelo:

$$\begin{array}{llll}y_i = \beta' X_i + u_i & &\text{ si} & RHS>0 \\ y_i = 0 & & \text{ de otra forma}\end{array}\tag{8.1}$$ 

Este problema de estimación de la ecuación (8.1) es que $E(u_i)\neq 0$ debido a trucar. La idea básica detras del método estimar el valor esperado de $E(u_i)$, sustituirlo en la ecuación (8.1) y estimar por ols, en lugar de Tobit.

Debido a que tenemos la variable truncada, tenemos que:

$$E\left(y_i|y_i>0\right) = \beta'X_i + E\left(u_i|u_i>-\beta'X\right)$$

Donde el valor esperado de los errores se puede representar como:

$$E(u_i|u_i>-\beta'X) = \sigma\frac{\phi}{\Phi}$$

Reemplazando nos queda:

$$E\left(y_i|y_i>0\right) = \beta'X_i + \sigma\frac{\phi}{\Phi} \tag{8.2}$$

Por lo tanto, podemos escribir a la ecuación (8.1) como:

$$y_i = \beta'X_i + \sigma \frac{\phi_i}{\Phi_i} + v_i \tag{8.3}$$

Lo que Heckman sugirió es lo sisguiente: Debido a que la función de probabilidad del modelo probit es bien definido, definimos la variable dummy como:

$$\begin{array}{llll}I_i = 1 & &\text{ si} & y_i>0 \\ I_i = 0 & & \text{ de otra forma}\end{array}$$ 

Por lo tanto, utilizando el modelo probit, obtenemos las estimaciones de $\beta/\sigma$. Utilizando estas obtenemos los valores de $\phi_i$ y $\Phi_i$. Y ahora, obtenemos estimaciones consistentes de $\beta$ y $\sigma$ utilizando (8.3) por OLS, utilizando $\hat \phi/\hat \Phi$ en lugar de $\phi/ \Phi$ como variables explicativas. 

Si utilizamos todas las observaciones $y_i$, y no solo las observaciones no cero como lo sugiere Heckman, tenemos:

$$E(y_i) = Prob(y_i>0)\cdot E(y_i|y_i>0) + Prob(y_i\leq 0) \cdot E(y_i|y_i\leq0)$$

$$E(y_i ) = \beta'\left(\Phi_iX_o\right) + \sigma\phi_i\tag{8.4}$$

Por lo tanto, luego de obtener las estimaciones de $\phi_i$ y $\Phi_i$, estimamos esta ecuación para obtener las estimaciones de $\beta$ y $\sigma$.

In [3]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
df = pd.read_stata('gsem_womenwk.dta')
df['censored'] = np.where((df['wage'] == 0) | (df['wage'].isna()), 0, 1)
df.head()

,age,educ,married,children,wage,censored
0,22,10,1,0,NaN,0
1,36,10,1,0,20.312845,1
2,28,10,1,0,NaN,0
3,37,10,1,0,NaN,0
4,39,10,1,1,16.142241,1


In [4]:
regprob =  smf.probit('censored ~ age + educ + married + children', data =df).fit()
print(regprob.summary())

Optimization terminated successfully.
         Current function value: 0.513531
         Iterations 6
                          Probit Regression Results                           
Dep. Variable:               censored   No. Observations:                 2000
Model:                         Probit   Df Residuals:                     1995
Method:                           MLE   Df Model:                            4
Date:                Tue, 15 Oct 2024   Pseudo R-squ.:                  0.1889
Time:                        16:15:20   Log-Likelihood:                -1027.1
converged:                       True   LL-Null:                       -1266.2
Covariance Type:            nonrobust   LLR p-value:                3.268e-102
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.4674      0.193    -12.813      0.000      -2.845      -2.090
age            0.0347      0.

In [5]:
beta = regprob.params[1:]
X = df[['age','educ','married','children']].to_numpy()

df['y'] = X@beta + regprob.params[0] #Calculo de errores. 

In [6]:
df.head()

,age,educ,married,children,wage,censored,y
0,22,10,1,0,NaN,0,-0.688997
1,36,10,1,0,20.312845,1,-0.202902
2,28,10,1,0,NaN,0,-0.480671
3,37,10,1,0,NaN,0,-0.168180
4,39,10,1,1,16.142241,1,0.348587


In [7]:
from scipy.stats import norm

df['phi'] = norm.pdf(df['y'])
df['Phi'] = regprob.predict() # es lo mismo que norm.cdf(df['y'])
df['Ee'] = df['phi']/df['Phi']

regols =  smf.ols('wage ~ age + educ + Ee', data =df).fit()
print(regols.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.279
Model:                            OLS   Adj. R-squared:                  0.278
Method:                 Least Squares   F-statistic:                     173.0
Date:                Tue, 15 Oct 2024   Prob (F-statistic):           8.63e-95
Time:                        16:15:25   Log-Likelihood:                -4158.2
No. Observations:                1343   AIC:                             8324.
Df Residuals:                    1339   BIC:                             8345.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7340      1.166      0.629      0.5

In [10]:
df['resid2'] = df['wage'] - df['predicted']

In [11]:
sigma = ((df['resid2']**2).sum()/(df['wage'].notna().sum() - 4))**0.5
sigma

5.358695656745396